In [1]:
# Imports
import os
import datetime

import pandas as pd
import numpy as np
from pyproj import CRS, Transformer

In [2]:
def ll2xy(lon, lat):
    """
    Transform coordinates from input geodetic coordinates (lon, lat)
    to output Antarctic Polar Stereographic coordinates (x, y)

    Parameters
    lon - Geodetic longitude in EPSG:4326 [float]
    lat - Geodetic latitude in EPSG:4326 [float]

    Returns
    x - Antarctic Polar Stereographic (EPSG:3031) x [float]
    y - Antarctic Polar Stereographic (EPSG:3031) y [float]
    """

    crs_ll = CRS("EPSG:4326")
    crs_xy = CRS("EPSG:3031")
    ll_to_xy = Transformer.from_crs(crs_ll, crs_xy, always_xy=True)
    x, y = ll_to_xy.transform(lon, lat)
    return x, y

In [ ]:
def load(file):
    """
    Load processed gps data file into pandas table.

    Parameters
    file - .pos precise point solution file, Natural Resources Canada [string]

    Returns
    data - Pandas DataFrame with the following columns, extracted from file
        time - Time as datetime object
        day_of_year - Time as julian day
    flip - flag to flip order of readings for 2024 csrs-ppp data
    """

    data = pd.DataFrame()  # Create Pandas DataFrame
    flip = False
    # Read data file

    # Convert longitude and latitude from deg min sec to fractional degrees
    # Three different file formats so try one first and try the other if it
    # throws a not found exception.

    try:
        # CSRS-PPP 2024
        d = pd.read_csv(file, skiprows=3, sep="\s+")
        data["longitude"] = d["LONDD"] - d["LONMN"] / 60 - d["LONSS"] / 60 / 60
        data["latitude"] = d["LATDD"] - d["LATMN"] / 60 - d["LATSS"] / 60 / 60
        data["time"] = pd.to_datetime(d["YEAR-MM-DD"] + "T" + d["HR:MN:SS.SS"])
        data["day_of_year"] = d["DAYofYEAR"]

        # Look at data and decide if to flip
        if len(data.index) > 1:
            diff = data["time"].iloc[0] - data["time"].iloc[1]
            if diff > datetime.timedelta(seconds=0):
                flip = True
    except:  # noqa: E722
        try:
            d = pd.read_csv(file, skiprows=7, delim_whitespace=True)
            data["longitude"] = d["LON(d)"] - d["LON(m)"] / 60 - d["LON(s)"] / 60 / 60
            data["latitude"] = d["LAT(d)"] - d["LAT(m)"] / 60 - d["LAT(s)"] / 60 / 60
            data["time"] = pd.to_datetime(d["YEAR-MM-DD"] + "T" + d["HR:MN:SS.SSS"])
            data["day_of_year"] = d["DOY"]
        except:  # noqa: E722
            try:
                d = pd.read_csv(file, skiprows=5, delim_whitespace=True)
                data["longitude"] = d["LONDD"] - d["LONMN"] / 60 - d["LONSS"] / 60 / 60
                data["latitude"] = d["LATDD"] - d["LATMN"] / 60 - d["LATSS"] / 60 / 60
                data["time"] = pd.to_datetime(d["YEAR-MM-DD"] + "T" + d["HR:MN:SS.SS"])
                data["day_of_year"] = d["DAYofYEAR"]
            except:  # noqa: E722
                d = pd.read_csv(file, skiprows=6, delim_whitespace=True)
                data["longitude"] = (
                    d["LON(d)"] - d["LON(m)"] / 60 - d["LON(s)"] / 60 / 60
                )
                data["latitude"] = (
                    d["LAT(d)"] - d["LAT(m)"] / 60 - d["LAT(s)"] / 60 / 60
                )
                data["time"] = pd.to_datetime(d["YEAR-MM-DD"] + "T" + d["HR:MN:SS.SSS"])
                data["day_of_year"] = d["DOY"]

    x, y = ll2xy(lon=data["longitude"], lat=data["latitude"])
    data["x"] = x
    data["y"] = y

    x0 = data["x"][0]
    y0 = data["y"][0]

    data["dist"] = np.sqrt((data["x"] - x0) ** 2 + (data["y"] - y0) ** 2)
    return data, flip

In [4]:
def datastream(dir, years):
    """
    Takes input directory dir with year subdirectories containign .pos files.
    Starting from the first file in the first year, append days until all files
    in the directory have been added.

    Parameters
    dir - Input directory tree with structure described in top comment [string]
    years - years to be run [arr of strings]

    Returns
    data - Resulting multiday time series, data. [DataFrame]
    """
    data = pd.DataFrame()
    for year in years:
        for folder in os.scandir(dir.path):
            if folder.is_dir():  # folder is an os.DirEntry object
                if folder.name == year:
                    print(folder.name)
                    for gps in os.listdir(folder.path):
                        if gps.endswith(".pos") and not gps.startswith(
                            "."
                        ):  # Ignore xyzt, zip files
                            ind_data, flip = load(folder.path + "/" + gps)
                            if flip:  # Reorder the data if using CSRS-PPP 2024 pre 2018
                                ind_data = ind_data.reindex(index=ind_data.index[::-1])
                            data = pd.concat([data, ind_data], ignore_index=True)
    return data

In [8]:
dir = "/mnt/d/csrs_2024/all"


stano = 0
for sta in os.scandir(dir):
    if sta.is_dir():
        stano = stano + 1
        print(sta.name, sta.path)
        if sta.name == "gz20":
            years = [
                "2006",
                "2007",
                "2008",
                "2009",
                "2010",
                "2011",
                "2012",
                "2013",
                "2014",
                "2015",
                "2016",
                "2017",
                "2018",
                "2019",
                "2020",
            ]
            # Make empty dataframe for each station to be filled in with starts/ends
            out = pd.DataFrame(columns=["x", "y", "start", "end", "station", "stano"])
            stream = datastream(sta, years)  # Load data
            x = stream["x"].iloc[0]
            y = stream["y"].iloc[0]
            # print(stream)
            start_time = stream["time"][0]
            for i in range(len(stream["time"])):
                if i > 0:
                    prior_time = stream["time"][i - 1]
                    current_time = stream["time"][i]

                    delta_t = current_time - prior_time
                    if delta_t > datetime.timedelta(
                        days=1, minutes=-1
                    ):  # Check if > 1 day
                        end_time = prior_time
                        # print(start_time)
                        # print(end_time)
                        out.loc[len(out.index)] = [
                            x,
                            y,
                            start_time,
                            end_time,
                            sta.name,
                            stano,
                        ]
                        start_time = current_time

                    if i == len(stream["time"]) - 1:
                        end_time = current_time

                        out.loc[len(out.index)] = [
                            x,
                            y,
                            start_time,
                            end_time,
                            sta.name,
                            stano,
                        ]

            print(out)

            out["start"] = out["start"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
            out["end"] = out["end"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
            out.to_csv(
                "sta_uptime_plot.txt",
                sep="\t",
                index=False,
                mode="a",
                header=not "sta_uptime_plot.txt",
            )

            with open("sta_uptime_plot.txt", "a") as file:
                file.write("\n")

la16 /mnt/d/csrs_2024/all/la16
la17 /mnt/d/csrs_2024/all/la17
la18 /mnt/d/csrs_2024/all/la18
slw1 /mnt/d/csrs_2024/all/slw1
ws04 /mnt/d/csrs_2024/all/ws04
ws05 /mnt/d/csrs_2024/all/ws05
gz01 /mnt/d/csrs_2024/all/gz01
gz02 /mnt/d/csrs_2024/all/gz02
gz03 /mnt/d/csrs_2024/all/gz03
gz04 /mnt/d/csrs_2024/all/gz04
gz05 /mnt/d/csrs_2024/all/gz05
gz06 /mnt/d/csrs_2024/all/gz06
gz07 /mnt/d/csrs_2024/all/gz07
gz08 /mnt/d/csrs_2024/all/gz08
gz09 /mnt/d/csrs_2024/all/gz09
gz10 /mnt/d/csrs_2024/all/gz10
gz11 /mnt/d/csrs_2024/all/gz11
gz12 /mnt/d/csrs_2024/all/gz12
gz13 /mnt/d/csrs_2024/all/gz13
gz14 /mnt/d/csrs_2024/all/gz14
gz15 /mnt/d/csrs_2024/all/gz15
gz16 /mnt/d/csrs_2024/all/gz16
gz17 /mnt/d/csrs_2024/all/gz17
gz18 /mnt/d/csrs_2024/all/gz18
gz19 /mnt/d/csrs_2024/all/gz19
gz20 /mnt/d/csrs_2024/all/gz20
2016


/tmp/ipykernel_1290/2913353795.py:38: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  d = pd.read_csv(file, skiprows = 7, delim_whitespace=True)
/tmp/ipykernel_1290/2913353795.py:45: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  d = pd.read_csv(file, skiprows = 5, delim_whitespace=True)
/tmp/ipykernel_1290/2913353795.py:51: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  d = pd.read_csv(file, skiprows = 6, delim_whitespace=True)


KeyError: 'LON(d)'

In [11]:
# Load File and get true times with no data

times = pd.read_csv("sta_uptime_input.txt", sep="\t", header=None)
print(times)

                               0                           1     2   3
0     2010-01-15T00:40:00.000000  2010-05-02T15:59:00.000000  la01   1
1     2010-11-09T20:18:45.000000  2011-05-19T22:59:30.000000  la01   1
2     2011-11-15T06:44:45.000000  2012-05-16T19:56:45.000000  la01   1
3     2012-09-08T23:02:30.000000  2012-09-09T01:45:00.000000  la01   1
4     2012-09-10T20:02:15.000000  2012-09-10T22:08:00.000000  la01   1
...                          ...                         ...   ...  ..
1329  2016-09-03T09:20:45.000000  2016-09-03T21:25:00.000000  mg04  45
1330  2016-09-05T21:14:30.000000  2016-11-26T20:57:00.000000  mg04  45
1331  2014-01-24T02:34:00.000000  2014-12-17T23:59:45.000000  mg05  46
1332  2014-01-23T22:39:00.000000  2014-12-18T03:15:30.000000  mg06  47
1333  2014-01-27T01:18:45.000000  2014-12-18T23:59:45.000000  mg07  48

[1334 rows x 4 columns]
